In [ ]:
import pandas as pd

## Caminhos

In [4]:
#caminho Dani
bruto = pd.read_csv(r'C:\JupyterLab\2° Semestre\Machine Learning\ML-2022-DIMP\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\JupyterLab\\2° Semestre\\Machine Learning\\ML-2022-DIMP\\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv'

In [ ]:
#caminho Isa
bruto = pd.read_csv()

In [ ]:
#caminho Marcos
bruto = pd.read_csv(r'C:\Users\marcos220057\OneDrive - CNPEM - Centro Nacional de Pesquisa em Energia e Materiais\2sem2022\Aprendizado de Máquina\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

In [ ]:
#caminho Paola
bruto = pd.read_csv(r'C:\JupyterLab\ML-2022-DIMP\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

## Primeira escolha de atributos:

In [ ]:
# Excluir as colunas que tem NaN
bruto_completo = bruto.dropna(axis = 1, how ='any')
del bruto_completo['file_name']
del bruto_completo['charge_state']

In [ ]:
#Calculado a matriz de correlação peterson para ver quais atributos são bons para energia de Fermi
bruto_completo.shape
corrmat = bruto_completo.corr()

In [5]:
#printando os atributos que correlacionam melhor que 0.782 para energia de fermi
Atr1 = []
Ef = bruto_completo['Fermi_energy']
for i in corrmat['Fermi_energy'].keys():
    if abs(corrmat['Fermi_energy'][i]) > 0.782:
        Atr1.append(i)
        print(i, 'correlation:', corrmat['Fermi_energy'][i])
        print()

NameError: name 'bruto_completo' is not defined

In [6]:
#printando os atributos que correlacionam melhor que 0.93 para energia total
Et = bruto_completo['total_energy']
Atr2 = []
for i in corrmat['total_energy'].keys():
    if abs(corrmat['total_energy'][i]) > 0.93:
        print(i, corrmat['total_energy'][i])
        Atr2.append(i)
        print()

NameError: name 'bruto_completo' is not defined

### Plotando os gráficos da regressão linear das features que relacionam com os targets:

In [ ]:
import lmfit 
import matplotlib.pyplot as plt

In [ ]:
def plotar(x, y, ux = 0, uy = 0, reta = False, legendas = ['x', 'y'], c='b', save = False, line = False):
    '''
    Essa função plota um gráfico de pontos ou de linha com ou sem barras de incerteza e com ou sem regressão linear.
    Argumentos obrigatórios:
    - x: Lista valores de x
    - y: Lista valores de y
    
    Argumentos opcionais:
    - ux: Lista com os valores de incertezas de x (dafault: ux = 0)
    - uy: Lista com os valores de incertezas de y (dafault: uy = 0)
    - reta: bool que define se quer regressão linear ou não (default: reta = False)
    - legendas: lista com duas strings correspondentes àa legendaa do eixo x e y (default: legendas = ['x', 'y'])
    - c: string da cor dos pontos (default: c = 'b')
    - save: string com o nome de um arquivo png, ou o bool False para não salvar (default: save = False)
    
    '''
    config = {'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.edgecolor': '.4',
    'axes.labelcolor': '.0',
    'axes.titlesize': 'large',
    'axes.labelsize': 'medium',
    'figure.autolayout': True,
    'figure.figsize': (4.5, 3.5),
    'font.family': ['serif'],
    'font.size': 10.0,
    'grid.linestyle': '--',
    'legend.facecolor': '.9',
    'legend.frameon': True,
    'savefig.transparent': True,
    'text.color': '.0',
    'xtick.labelsize': 'small',
    'ytick.labelsize': 'small',}

    plt.style.use(['seaborn-whitegrid', 'seaborn-paper', 'seaborn-muted', config])
    fig,ax1 = plt.subplots(1,1)
    ax1.tick_params(direction='in', which='both',top=True,right=True,labelsize=12)
    
    #Labels
    ax1.set_ylabel(legendas[1], fontsize=12)
    ax1.set_xlabel(legendas[0], fontsize=12)
    #regressão linear
    if reta == True:
        def reta(x, a, b):
            return a*x + b

        reta_model=lmfit.Model(reta)

        #se der errado tentar mexer nesses valores do chute inicial:
        
        reta_model.set_param_hint('a',value=1,min= -1000000,max=3000000,vary=True) 
        reta_model.set_param_hint('b',value=1,min=-200000,max=2000000,vary=True) 
        param = reta_model.make_params()

        results = reta_model.fit(y, x = x, params = param)
        #print(results.fit_report())
        ax1.plot(x, y, 'o', markersize=0.3)
        ax1.plot(x,results.best_fit,label="Regressão Linear", c = 'r', linewidth=2) ; ax1.legend(fontsize='small')
    if line == True:
        ax1.plot(x, y)
    elif ux == 0 and uy == 0:
        ax1.plot(x, y, 'bo')

    else:
        ax1.errorbar(x, y, yerr=uy, xerr=ux, fmt='o',elinewidth=1,capsize=3,capthick=1,ms=3,c=c,ecolor='black')
    #save
    if save != False:
        plt.tight_layout()
        plt.savefig(save, dpi=200)
        plt.show()

### Energia de Fermi

In [ ]:
#Atr1.remove(Atr1[-1])
for i in Atr1:
    plotar(bruto_completo[i], Ef, legendas = [i, 'Fermi energy'],reta = True)

In [ ]:
#Atr2.remove(Atr2[-1])
for i in Atr2:
    plotar(bruto_completo[i], Et, legendas = [i, 'total_energy'], reta = True)

In [ ]:
dataEf = {}
for i in Atr1:
    dataEf.update({i:bruto[i]})
dataEf = pd.DataFrame(dataEf)

In [ ]:
import seaborn as sns

In [7]:
# Plotar o gráfico de correlações entre as features que relacionam com fermi
corrmat1 = dataEf.corr()
f, ax = plt.subplots(figsize =(16, 9))
sns.heatmap(corrmat1, ax = ax, cmap ="BrBG", linewidths = 1, vmin=-1, vmax=1, annot=True)

NameError: name 'dataEf' is not defined

In [ ]:
dataEt = {}
for i in Atr2:
    dataEt.update({i:bruto[i]})
dataEt = pd.DataFrame(dataEt)

In [ ]:
corrmat1 = dataEt.corr()
f, ax = plt.subplots(figsize =(16, 9))
sns.heatmap(corrmat1, ax = ax, cmap ="BrBG", linewidths = 1, vmin=-1, vmax=1, annot=True)

In [ ]:
del bruto_completo['Shape']

In [ ]:
import numpy as np

In [ ]:
atributos_1 = {}
for i in bruto_completo.keys():
    if np.isfinite(corrmat['Fermi_energy'][i]):
        atributos_1.update({i : abs(corrmat['Fermi_energy'][i])})

In [ ]:
L = sorted(atributos_1,key = atributos_1.get,  reverse = True)
L.remove('Fermi_energy')
print(L[0])

In [ ]:
L = L[0:100]
L1 = L.copy()
for i in L:
    for k in L:
        if i < k and abs(corrmat[i][k])>0.8:
            if abs(corrmat['Fermi_energy'][i]) > abs(corrmat['Fermi_energy'][k]) and k in L1 and i in L1:
                L1.remove(k)
            elif k in L1 and i in L1:
                L1.remove(i)

In [ ]:
#Esses serão os atributos para calcular a energia de Fermi (resta saber o que eles significam
for i in L1:
    print(i, ' correlation:',corrmat['Fermi_energy'][i])
    print()

In [ ]:
M = {}
M.update({'Fermi_energy':bruto['Fermi_energy']})
for i in L1:
    M.update({i:bruto[i]})
M = pd.DataFrame(M)
corrmat_F = M.corr()
f, ax = plt.subplots(figsize =(16, 9))
sns.heatmap(corrmat_F, ax = ax, cmap ="BrBG", linewidths = 1, vmin=-1, vmax=1, annot=True)
plt.savefig('atributos_energia_fermi.png', dpi = 200)

In [ ]:
Dados_Fermi = M.copy()

In [ ]:
Dados_Fermi.to_csv(r"C:\JupyterLab\meu-jupyter-ilum\2° semestre\AM\ML-2022-DIMP\Dados_Fermi.csv")

### Energia total

In [ ]:
atributos_2 = {}
for i in bruto_completo.keys():
    if np.isfinite(corrmat['total_energy'][i]):
        atributos_2.update({i : abs(corrmat['total_energy'][i])})

In [ ]:
L2 = sorted(atributos_2,key = atributos_2.get,  reverse = True)
L2.remove('total_energy')
print(L2[0])

In [ ]:
L2 = L2[0:100]
Lb = L2.copy()
for i in L2:
    for k in L2:
        if i < k and abs(corrmat[i][k])>0.7:
            if abs(corrmat['total_energy'][i]) > abs(corrmat['total_energy'][k]) and k in Lb and i in Lb:
                Lb.remove(k)
            elif k in Lb and i in Lb:
                Lb.remove(i)

In [ ]:
#Esses serão os atributos para calcular a energia de Fermi (resta saber o que eles significam)
for i in Lb:
    print(i, ' correlation:',corrmat['total_energy'][i])
    print()

#### Como observamos pelos gráficos a única coisa que relacionava com a energia total do flake é o número de átomos. Por isso, preferimos mudar a nossa abordagem e utilizar como target a energia por átomos.

## Segunda tentativa (energia por átomo):

In [ ]:
# Função para criar lista (p/ cada exemplo ) do número de átomos e verificar se existem NaNs:
E = []
i = 0
while i < len(bruto_completo['C']):
    E.append(bruto_completo['total_energy'][i]/bruto_completo['atom_number_total'][i])
    i = i+1
print(len(bruto_completo['total_energy']), len(E))

In [8]:
Dados = bruto_completo.assign(energy_per_atom = E) # Acrescentando uma coluna que nos dá E = a energia por átomos. Que consiste na divisão da energia total do flake pelo n° de átomo

NameError: name 'bruto_completo' is not defined

In [9]:
plotar(Dados['atom_number_total'], Dados['energy_per_atom'], legendas = ['atom_number_total', 'energy_per_atom'], reta = True)
#A energia agora não está totalmente dependente do número de átomos

NameError: name 'plotar' is not defined

In [ ]:
#Cria a matriz de correlação 
Dados.shape
corrmat = Dados.corr()

In [ ]:
#Pegar as colunas em que foi possível calcular a correlação
atributos_3 = {}
for i in Dados.keys():
    if np.isfinite(corrmat['energy_per_atom'][i]): #Se não for Nan, adiciona a atributos_2
        atributos_2.update({i : abs(corrmat['energy_per_atom'][i])})

In [ ]:
#Selecionar os atributos que tem uma alta correlação, exceto aqueles em que a correlação é 1
L3 = sorted(atributos_2,key = atributos_2.get,  reverse = True)
L3.remove('energy_per_atom')
L3.remove('Fermi_energy')
print(L3[0])

In [ ]:
L3 = L3[0:100]
Lb = L3.copy()
for i in L3:
    for k in L3:
        if i < k and abs(corrmat[i][k])>0.76:
            if abs(corrmat['energy_per_atom'][i]) > abs(corrmat['energy_per_atom'][k]) and k in Lb and i in Lb:
                Lb.remove(k)
            elif k in Lb and i in Lb:
                Lb.remove(i)

In [ ]:
#Esses serão os atributos para calcular a energia de Fermi (resta saber o que eles significam)
for i in Lb:
    print(i, ' correlation:',corrmat['energy_per_atom'][i])
    print()

In [ ]:
M = {}
M.update({'energy_per_atom':Dados['energy_per_atom']})
for i in Lb:
    M.update({i:bruto[i]})
M = pd.DataFrame(M)
corrmat_F = M.corr()
f, ax = plt.subplots(figsize =(16, 9))
sns.heatmap(corrmat_F, ax = ax, cmap ="BrBG", linewidths = 1, vmin=-1, vmax=1, annot=True)
plt.savefig('atributos_energia_por_atomo.png', dpi = 200)

In [ ]:
Dados_energia = M.copy()

In [10]:
Dados_energia.to_csv(r"C:\JupyterLab\ML-2022-DIMP\CSVs\Dados_energia.csv")
#Pegamos todos os dados que escolhemos através das correlações e criamos um novo csv

NameError: name 'Dados_energia' is not defined

#### Vamos agora utilizar os Data Frames com os dados que nós realmente vamos utilizar, de energia de Fermi e por átomo

In [ ]:
#caminho pc paola
df_fermi = pd.read_csv (r"C:\JupyterLab\ML-2022-DIMP\CSVs\Dados_Fermi.csv")
df_energia= pd.read_csv(r"C:\JupyterLab\ML-2022-DIMP\CSVs\Dados_energia.csv")

In [ ]:
#caminho pc isadora
df_energia= pd.read_csv(r"C:\JupyterLab\ML-2022-DIMP\CSVs\Dados_energia.csv")
df_fermi= pd.read_csv(r"C:\JupyterLab\ML-2022-DIMP\CSVs\Dados_Fermi.csv")

### Tarefa de análise exploratória dos dados

In [ ]:
print(df_fermi.dtypes,df_energia.dtypes)

In [ ]:
# dados da energia de fermi
print( df_fermi.describe())

In [11]:
#dados da energia por átomo
print(df_energia.describe())

NameError: name 'df_energia' is not defined

In [ ]:
def histograma(x, density=False, c = 'r', bins = 'sqrt', save = False, legendas = ['x', 'Frequência'], title = 'Histograma'):
    '''
    Plota um histograma a partir de uma lista de dados.
    Argumentos obrigatórios:
    - x: Lista dos valores da medição/dados
    
    Argumentos opcionais:
    - Density: bool que, se for True, faz com que o histograma mostre densidade de probabilidade (default: density=False)
    - c: string para a cor das barras do histograma (default: c = 'r')
    - bins: método de determinação do tamanho de intervalos do histograma (default: bins = 'sqrt')
    - save: string com o nome do arquivo png para salvar o grafico. Se save = False, então não irá salvar. (default: save = False)
    - legendas: lista das strings de legenda dos eixos (default: legendas = ['x', 'Frequência'])
    
    '''
    config = {'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.edgecolor': '.4',
    'axes.labelcolor': '.0',
    'axes.titlesize': 'large',
    'axes.labelsize': 'medium',
    'figure.autolayout': True,
    'figure.figsize': (4.5, 3.5),
    'font.family': ['serif'],
    'font.size': 10.0,
    'grid.linestyle': '--',
    'legend.facecolor': '.9',
    'legend.frameon': True,
    'savefig.transparent': True,
    'text.color': '.0',
    'xtick.labelsize': 'small',
    'ytick.labelsize': 'small',}
    
    plt.style.use(['seaborn-whitegrid', 'seaborn-paper', 'seaborn-muted', config])
    fig,axh = plt.subplots(1,1)
    H = plt.hist(x,bins = bins, color = c,density = density)
    axh.set_title(title)
    axh.set_ylabel(legendas[1],fontsize=12)
    axh.set_xlabel(legendas[0],fontsize=12)
    if save != False:
        plt.savefig(save, dpi=200)

In [ ]:
histograma(bruto_completo['Fermi_energy'], legendas = ['Fermi energy', 'frequência'],title = 'Histograma Energia de Fermi', save = 'histograma_fermi.png')

In [ ]:
histograma(M['energy_per_atom'], legendas = ['Energy per atom', 'frequência'],title = 'Histograma Energia por átomo', save = 'histograma_energia.png')